<a href="https://colab.research.google.com/github/urmilapol/urmilapolprojects/blob/master/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===== COMPLETE STOCK P/E RATIO ETL PIPELINE =====
# Video: https://www.youtube.com/watch?v=KAuIvccwbPY

# 1. Initialize SparkSession (Driver Program Entry Point)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg, count

# Create SparkSession - connects to cluster (local/4 nodes/serverless)
# Use cluster: "yarn", "k8s://...", or omit for default
spark = SparkSession.builder \
    .appName("StockPEratioDemo") \
    .master("local[*]") \
    .getOrCreate()

print("✅ SparkSession created. Cluster ready for distributed processing.")

# 2. EXTRACT: Load CSV (Spark auto-partitions across workers)
df_raw = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/content/sample_data/stocks.csv")  # Scales to HDFS/S3 paths

print("📊 Raw data schema:")
df_raw.printSchema()
df_raw.show()

# 3. TRANSFORM: Business logic (lazy evaluation - no execution yet)
df_pe = df_raw.withColumn(
    "pe_ratio",
    col("Price") / col("EPS")
).withColumn(
    "pe_category",
    when(col("pe_ratio") < 15, "Undervalued")
    .when(col("pe_ratio") > 30, "Overvalued")
    .otherwise("Fair")
)

print("🔄 Transformation DAG built (lazy)")

# 4. LOAD/ACTION: Triggers execution across cluster
print("🚀 EXECUTING across cluster...")
df_result = df_pe.orderBy(col("pe_ratio"))
df_result.select("Symbol", "Price", "EPS", "pe_ratio", "pe_category").show(truncate=False)

# 5. Advanced: Aggregate analytics (real ETL use case)
df_summary = df_pe.groupBy("pe_category").agg(
    avg("pe_ratio").alias("avg_pe"),
    count("Symbol").alias("stock_count")
).orderBy("pe_category")
print("📈 Portfolio Summary:")
df_summary.show()

# 6. Write results (complete ETL)
df_result.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("stocks_pe_analysis")

print("💾 Results written to stocks_pe_analysis/")

# 7. Cleanup
spark.stop()
print("✅ Spark cluster shutdown.")

✅ SparkSession created. Cluster ready for distributed processing.
📊 Raw data schema:
root
 |-- Symbol: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- EPS: double (nullable = true)

+------+-------+-----+
|Symbol|  Price|  EPS|
+------+-------+-----+
|  AAPL|  175.5| 6.16|
|  GOOG|2850.25|145.6|
|  MSFT|  425.8| 11.8|
|  TSLA|  245.3| 3.65|
|  AMZN|  186.8| 4.12|
|  META|  567.9|20.35|
|  NVDA| 890.45|12.05|
+------+-------+-----+

🔄 Transformation DAG built (lazy)
🚀 EXECUTING across cluster...
+------+-------+-----+------------------+-----------+
|Symbol|Price  |EPS  |pe_ratio          |pe_category|
+------+-------+-----+------------------+-----------+
|GOOG  |2850.25|145.6|19.575892857142858|Fair       |
|META  |567.9  |20.35|27.906633906633903|Fair       |
|AAPL  |175.5  |6.16 |28.490259740259738|Fair       |
|MSFT  |425.8  |11.8 |36.08474576271186 |Overvalued |
|AMZN  |186.8  |4.12 |45.33980582524272 |Overvalued |
|TSLA  |245.3  |3.65 |67.2054794520548  |Overvalu

https://www.chaosgenius.io/blog/apache-spark-architecture/


Apache Spark follows a master-slave (driver-worker) architecture for distributed data processing, ideal for data engineering pipelines and interviews.
High-Level Architecture
Spark operates with a Driver Program (master), Cluster Manager, and Executors (workers) on cluster nodes. Tasks represent the smallest unit: a Spark job divides into stages, stages into tasks executed in parallel.[2][1]
Cluster Manager (Standalone, YARN, Mesos, Kubernetes) allocates CPU/memory resources dynamically, upscaling for faster completion.[1]
Master-slave: Driver coordinates; executors process; fault-tolerant via recomputation.[1]
Driver Responsibilities
Driver runs the main Spark application (your PySpark code), converting user code into a Directed Acyclic Graph (DAG)—a one-way graph of operations without loops. Requests resources from Cluster Manager, schedules tasks on executors, tracks progress, and collects results.[1]
Key interview point: Driver handles logical-to-physical plan translation via DAG Scheduler.[1]
Cluster Manager Role
Manages resource allocation (e.g., CPUs, memory) to executors, monitors availability, and assigns free resources. YARN (Yet Another Resource Negotiator) is common in Hadoop ecosystems.[2][1]
Executors and Tasks
Executors (worker processes on nodes) execute tasks, store data in memory/disk, with dedicated CPU cores and task slots. Each holds executor memory for caching.[1]
Task Scheduler assigns tasks to executors post-DAG breakdown.[1]
Job Execution Flow
•	User submits code → Driver creates logical plan.
•	DAG Scheduler: Logical → physical plan, breaks job into stages (shuffle boundaries).
•	Task Scheduler: Stages → tasks assigned to executors.
•	Cluster Manager allocates resources.
•	Executors run tasks in parallel → Driver aggregates results for storage/output.[1]



In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import upper

spark = SparkSession.builder.appName("DataTransformation").getOrCreate()

# Sample data: courses with fees and discounts
data = [("Java", 4000, 5), ("Python", 4600, 10), ("Scala", 4100, 15)]
columns = ["CourseName", "fee", "discount"]
df = spark.createDataFrame(data, columns)
df.show(truncate=False)


+----------+----+--------+
|CourseName|fee |discount|
+----------+----+--------+
|Java      |4000|5       |
|Python    |4600|10      |
|Scala     |4100|15      |
+----------+----+--------+



In [2]:
def to_upper(df):
    return df.withColumn("CourseName", upper(df.CourseName))

def reduce_price(df, amount):
    return df.withColumn("new_fee", df.fee - amount)

def apply_discount(df):
    return df.withColumn("discounted_fee", df.new_fee * (1 - df.discount / 100))

# Apply chain
result = df.transform(to_upper).transform(reduce_price, 1000).transform(apply_discount)
result.select("CourseName", "discounted_fee").show()


+----------+--------------+
|CourseName|discounted_fee|
+----------+--------------+
|      JAVA|        2850.0|
|    PYTHON|        3240.0|
|     SCALA|        2635.0|
+----------+--------------+



**Apache Spark excels in real-world data processing through transformations like joins, aggregations, and window functions on large datasets. Here's another hands-on PySpark example focused on sales data aggregation—a common ETL scenario for retail analytics that builds on the prior fee transformation demo**
Sales Aggregation Example
# This processes transactional sales data to compute daily revenue by product category, filtering invalid records and applying windowed ranking for top performers.
This PySpark code sets up an ETL (Extract, Transform, Load) pipeline to process sales data, aggregate it by daily revenue per category, and then rank the categories within each day.



In [9]:
from pyspark.sql import SparkSession
# It imports necessary classes and functions from pyspark.sql  which is a library for working with structured data in Spark. It includes DataFrame for various data operations, and Window for defining window functions.
from pyspark.sql.functions import col, to_date, sum as spark_sum, rank, desc
from pyspark.sql.window import Window
#creates or retrieves a SparkSession, which is the entry point to Spark functionality
spark = SparkSession.builder.appName("SalesAggregation").getOrCreate()

# Sample sales data (scale to CSV from Kaggle e-commerce datasets)
#sales_data is a Python list of tuples representing raw sales records, and columns defines the schema for this data. df = spark.createDataFrame(sales_data, columns) then converts this Python data into a Spark DataFrame.
sales_data = [
    ("2025-01-01", "Electronics", 100, 2),
    ("2025-01-01", "Clothing", 50, 5),
    ("2025-01-02", "Electronics", 100, 1),
    ("2025-01-02", "Clothing", 50, 3),
    ("2025-01-01", "Books", 20, 10),  # Low price, high volume
    ("2025-01-03", "Books", 20, 0)    # Invalid (zero qty)
]

columns = ["sale_date", "category", "price", "quantity"]
df = spark.createDataFrame(sales_data, columns)

# ETL Pipeline: Clean → Transform → Aggregate
#converts the sale_date column from a string to a proper date type.
df_clean = df.filter(col("quantity") > 0).withColumn("sale_date", to_date(col("sale_date")))
#aggregates the cleaned data: groups by sale_date and category, and calculates the total revenue for each combination. calculates the total revenue for each group by multiplying price and quantity and summing them up.
df_agg = df_clean.groupBy("sale_date", "category").agg(spark_sum(col("price") * col("quantity")).alias("revenue"))

# Window function for ranking top categories per day   This section calculates the rank of each category's revenue within each day
#defines a window specification. It partitions the data by sale_date (meaning ranks are calculated independently for each day) and orders the results within each partition by revenue in descending order.
#applies this window function to df_agg, creating a new column named rank that assigns a rank to each category based on its revenue within its respective day.
window_spec = Window.partitionBy("sale_date").orderBy(desc("revenue"))
df_ranked = df_agg.withColumn("rank", rank().over(window_spec))
#displays the final df_ranked DataFrame, ordered by sale_date and then by revenue in descending order, showing the top-performing categories for each day.

df_ranked.orderBy("sale_date", desc("revenue")).show(truncate=False)

+----------+-----------+-------+----+
|sale_date |category   |revenue|rank|
+----------+-----------+-------+----+
|2025-01-01|Clothing   |250    |1   |
|2025-01-01|Electronics|200    |2   |
|2025-01-01|Books      |200    |2   |
|2025-01-02|Clothing   |150    |1   |
|2025-01-02|Electronics|100    |2   |
+----------+-----------+-------+----+



The complete PySpark stock P/E ratio ETL pipeline demonstrates Spark's distributed computing from extract → transform → load. Each line builds toward parallel execution across a cluster while abstracting complexity from the developer.[1]

In [11]:
# ===== COMPLETE STOCK P/E RATIO ETL PIPELINE =====
# Video: https://www.youtube.com/watch?v=KAuIvccwbPY

# 1. Initialize SparkSession (Driver Program Entry Point)
#: Imports core Spark SQL components. SparkSession creates the driver entry point; col() references DataFrame columns in expressions.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg, count

# Create SparkSession - connects to cluster (local/4 nodes/serverless)
# Use cluster: "yarn", "k8s://...", or omit for default
#Purpose: Creates driver program connecting to cluster.
#•	builder: Fluent API for configuration
#•	appName: Identifies job in Spark UI/Cluster Manager
#•	master("local[*]"): Uses all CPU cores locally (change to "yarn" for cluster)
#•	getOrCreate(): Singleton pattern - reuses existing session or creates new one. Result: Driver ready to orchestrate workers.
spark = SparkSession.builder \
    .appName("StockPEratioDemo") \
    .master("local[*]") \
    .getOrCreate()

print("✅ SparkSession created. Cluster ready for distributed processing.")



# ETL Step 1 - Distributed read operation.
#•	spark.read: DataFrameReader for structured formats
#•	header="true": First row becomes column names
#•	inferSchema="true": Auto-detects types (string→double)
#•	Key: Spark partitions file across executors automatically
#Lazy: No data loaded yet - just logical plan created.

# 2. EXTRACT: Load CSV (Spark auto-partitions across workers)
df_raw = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/content/sample_data/stocks.csv")  # Scales to HDFS/S3 paths

print("📊 Raw data schema:")
#Purpose: Actions trigger first execution.
#•	printSchema(): Shows inferred types (double for Price/EPS)
#•	show(): Materializes top 20 rows to driver console
#Triggers: Catalyst optimizer + DAG execution across cluster.

df_raw.printSchema()
df_raw.show()



#Purpose: ETL Step 2 - Business transformations (lazy).
#•	First withColumn: Creates pe_ratio = Price ÷ EPS
#•	Second withColumn: Conditional logic chains (when/otherwise)
#•	Immutable: Each returns new DataFrame
#•	Broadcast: Formulas applied in parallel on each partition
#DAG Built: Logical plan grows: read → divide → when() → when()



# 3. TRANSFORM: Business logic (lazy evaluation - no execution yet)
df_pe = df_raw.withColumn(
    "pe_ratio",
    col("Price") / col("EPS")
).withColumn(
    "pe_category",
    when(col("pe_ratio") < 15, "Undervalued")
    .when(col("pe_ratio") > 30, "Overvalued")
    .otherwise("Fair")
)

print("🔄 Transformation DAG built (lazy)")



#Purpose: Triggers full execution pipeline.
#•	orderBy: Shuffle stage (data crosses partitions)
#•	select: Column projection optimization
#•	show(truncate=False): Full column display
#Physical Plan: Driver → DAG Scheduler → Task Scheduler → Executors.


# 4. LOAD/ACTION: Triggers execution across cluster
print("🚀 EXECUTING across cluster...")
df_result = df_pe.orderBy(col("pe_ratio"))
df_result.select("Symbol", "Price", "EPS", "pe_ratio", "pe_category").show(truncate=False)


#Purpose: Real-world ETL aggregation pattern.
#•	groupBy: Shuffle by pe_category
#•	agg: Multiple aggregations in single pass
#•	alias: Renames output columns
#Optimization: Catalyst combines with prior operations.


# 5. Advanced: Aggregate analytics (real ETL use case)
df_summary = df_pe.groupBy("pe_category").agg(
    avg("pe_ratio").alias("avg_pe"),
    count("Symbol").alias("stock_count")
).orderBy("pe_category")
print("📈 Portfolio Summary:")
df_summary.show()



#Purpose: ETL Step 3 - Persist transformed data.
#•	coalesce(1): Single output file (remove for partitioned writes)
#•	mode("overwrite"): Replace existing output
#•	Distributed Write: Executors write parallel partitions.

# 6. Write results (complete ETL)
df_result.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("stocks_pe_analysis")

print("💾 Results written to stocks_pe_analysis/")


#Purpose: Release cluster resources (executors, memory).
#Execution Timeline
#1. Code written → Logical Plan (lazy transformations)
#2. .show()/.write() → Catalyst Optimization → Physical Plan
#3. DAG Scheduler → Stages → Tasks → Executors (parallel)
#4. Results → Driver → Console/Storage

# 7. Cleanup
spark.stop()
print("✅ Spark cluster shutdown.")

✅ SparkSession created. Cluster ready for distributed processing.
📊 Raw data schema:
root
 |-- Symbol: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- EPS: double (nullable = true)

+------+-------+-----+
|Symbol|  Price|  EPS|
+------+-------+-----+
|  AAPL|  175.5| 6.16|
|  GOOG|2850.25|145.6|
|  MSFT|  425.8| 11.8|
|  TSLA|  245.3| 3.65|
|  AMZN|  186.8| 4.12|
|  META|  567.9|20.35|
|  NVDA| 890.45|12.05|
+------+-------+-----+

🔄 Transformation DAG built (lazy)
🚀 EXECUTING across cluster...
+------+-------+-----+------------------+-----------+
|Symbol|Price  |EPS  |pe_ratio          |pe_category|
+------+-------+-----+------------------+-----------+
|GOOG  |2850.25|145.6|19.575892857142858|Fair       |
|META  |567.9  |20.35|27.906633906633903|Fair       |
|AAPL  |175.5  |6.16 |28.490259740259738|Fair       |
|MSFT  |425.8  |11.8 |36.08474576271186 |Overvalued |
|AMZN  |186.8  |4.12 |45.33980582524272 |Overvalued |
|TSLA  |245.3  |3.65 |67.2054794520548  |Overvalu